In [1]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions
library(SDMTools)
library(gbm)
library(randomForest)
library(zipcode)
library(mltools)
library(data.table)
library(caret)

#devtools::install_github("brews/bayfoxr", lib = "/kaggle/working")
install.packages("tidycensus")
library(tidycensus)

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.3     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   1.0.0          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loaded gbm 2.1.5

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin



Attaching package: ‘mltools’


The following object is masked from ‘package:tidyr’:

    replace_na



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required

[1] "agriculture-loan-prediction"

In [2]:
#read in files
train=read.csv("../input/agriculture-loan-prediction/train_comp1_2020.csv",stringsAsFactors=FALSE)
test=read.csv("../input/agriculture-loan-prediction/test_comp1_2020.csv",stringsAsFactors=FALSE)

In [3]:
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}

#Remove the nulls
#head(train$TYCornUnits[!is.numeric(train$TYCornUnits)])

train$TYCornUnits=abs(as.numeric(train$TYCornUnits))
train$TYCornUnits[is.na(train$TYCornUnits)]=median(train$TYCornUnits[!is.na(train$TYCornUnits)])
train$TYBeanUnits=as.numeric(train$TYBeanUnits)
train$TYBeanUnits[is.na(train$TYBeanUnits)]=median(train$TYBeanUnits[!is.na(train$TYBeanUnits)])
train$TYWheatUnits=as.numeric(train$TYWheatUnits)
train$TYWheatUnits[is.na(train$TYWheatUnits)]=median(train$TYWheatUnits[!is.na(train$TYWheatUnits)])
train$LYCornUnits=as.numeric(train$LYCornUnits)
train$LYCornUnits[is.na(train$LYCornUnits)]=median(train$LYCornUnits[!is.na(train$LYCornUnits)])
train$LYBeanUnits=as.numeric(train$LYBeanUnits)
train$LYBeanUnits[is.na(train$LYBeanUnits)]=median(train$LYBeanUnits[!is.na(train$LYBeanUnits)])
train$LYWheatUnits=as.numeric(train$LYWheatUnits)
train$LYWheatUnits[is.na(train$LYWheatUnits)]=median(train$LYWheatUnits[!is.na(train$LYWheatUnits)])
train$AT28=as.numeric(train$AT28)
train$AT28[is.na(train$AT28)]=median(train$AT28[!is.na(train$AT28)])
train$AT33=as.numeric(train$AT33)
train$AT33[is.na(train$AT33)]=median(train$AT33[!is.na(train$AT33)])
train$AT36=as.numeric(train$AT36)
train$AT36[is.na(train$AT36)]=median(train$AT36[!is.na(train$AT36)])
train$BC33=as.numeric(train$BC33)
train$BC33[is.na(train$BC33)]=median(train$AT36[!is.na(train$BC33)])
train$BC98=as.numeric(train$BC98)
train$BC98[is.na(train$BC98)]=median(train$BC98[!is.na(train$BC98)])
train$G068=as.numeric(train$G068)
train$G068[is.na(train$G068)]=median(train$G068[!is.na(train$G068)])
train$G091=as.numeric(train$G091)
train$G091[is.na(train$G091)]=median(train$G091[!is.na(train$G091)])
train$G093=as.numeric(train$G093)
train$G093[is.na(train$G093)]=median(train$G093[!is.na(train$G093)])
train$G094=as.numeric(train$G094)
train$G094[is.na(train$G094)]=median(train$G094[!is.na(train$G094)])
train$G096=as.numeric(train$G096)
train$G096[is.na(train$G096)]=median(train$G096[!is.na(train$G096)])
train$MT28=as.numeric(train$MT28)
train$MT28[is.na(train$MT28)]=median(train$MT28[!is.na(train$MT28)])
train$MT36=as.numeric(train$MT36)
train$MT36[is.na(train$MT36)]=median(train$MT36[!is.na(train$MT36)])
train$RE34=as.numeric(train$RE34)
train$RE34[is.na(train$RE34)]=median(train$RE34[!is.na(train$RE34)])
train$S063=as.numeric(train$S063)
train$S063[is.na(train$S063)]=median(train$S063[!is.na(train$S063)])
train$Zip_Code=as.numeric(train$Zip_Code)
train$Zip_Code[is.na(train$Zip_Code)]=getmode(train$Zip_Code[!is.na(train$Zip_Code)])

#train$TYCornUnits=log(abs(train$TYCornUnits+1))
#train$TYBeanUnits=log(abs(train$TYBeanUnits+1))
#train$TYWheatUnits=log(abs(train$TYWheatUnits+1))
#train$LYCornUnits=log(abs(train$LYCornUnits+1))
#train$LYBeanUnits=log(abs(train$LYBeanUnits+1))
#train$LYWheatUnits=log(abs(train$LYWheatUnits+1))
#train$AT28=log(abs(train$AT28+1))
#train$AT33=log(abs(train$AT33+1))
#train$AT36=log(abs(train$AT36+1))
#train$BC33=log(abs(train$BC33+1))
#train$BC98=log(abs(train$BC98+1))
#train$MT28=log(abs(train$MT28+1))
#train$MT36=log(abs(train$MT36+1))
#train$RE34=log(abs(train$RE34+1))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(e

In [4]:
test$TYCornUnits=abs(as.numeric(test$TYCornUnits))
test$TYCornUnits[is.na(test$TYCornUnits)]=median(train$TYCornUnits)
test$TYBeanUnits=as.numeric(test$TYBeanUnits)
test$TYBeanUnits[is.na(test$TYBeanUnits)]=median(train$TYBeanUnits)
test$TYWheatUnits=as.numeric(test$TYWheatUnits)
test$TYWheatUnits[is.na(test$TYWheatUnits)]=median(train$TYWheatUnits)
test$LYCornUnits=as.numeric(test$LYCornUnits)
test$LYCornUnits[is.na(test$LYCornUnits)]=median(train$LYCornUnits)
test$LYBeanUnits=as.numeric(test$LYBeanUnits)
test$LYBeanUnits[is.na(test$LYBeanUnits)]=median(train$LYBeanUnits)
test$LYWheatUnits=as.numeric(test$LYWheatUnits)
test$LYWheatUnits[is.na(test$LYWheatUnits)]=median(train$LYWheatUnits)
test$AT28=as.numeric(test$AT28)
test$AT28[is.na(test$AT28)]=median(train$AT28)
test$AT33=as.numeric(test$AT33)
test$AT33[is.na(test$AT33)]=median(train$AT33)
test$AT36=as.numeric(test$AT36)
test$AT36[is.na(test$AT36)]=median(train$AT36)
test$BC33=as.numeric(test$BC33)
test$BC33[is.na(test$BC33)]=median(train$BC33)
test$BC98=as.numeric(test$BC98)
test$BC98[is.na(test$BC98)]=median(train$BC98)
test$G068=as.numeric(test$G068)
test$G068[is.na(test$G068)]=median(train$G068)
test$G091=as.numeric(test$G091)
test$G091[is.na(test$G091)]=median(train$G091)
test$G093=as.numeric(test$G093)
test$G093[is.na(test$G093)]=median(train$G093)
test$G094=as.numeric(test$G094)
test$G094[is.na(test$G094)]=median(train$G094)
test$G096=as.numeric(test$G096)
test$G096[is.na(test$G096)]=median(train$G096)
test$MT28=as.numeric(test$MT28)
test$MT28[is.na(test$MT28)]=median(train$MT28)
test$MT36=as.numeric(test$MT36)
test$MT36[is.na(test$MT36)]=median(train$MT36)
test$RE34=as.numeric(test$RE34)
test$RE34[is.na(test$RE34)]=median(train$RE34)
test$S063=as.numeric(test$S063)
test$S063[is.na(test$S063)]=median(train$S063)


#test$TYCornUnits=log(abs(test$TYCornUnits+.01))
#test$TYBeanUnits=log(abs(test$TYBeanUnits+.01))
#test$TYWheatUnits=log(abs(test$TYWheatUnits+.01))
#test$LYCornUnits=log(abs(test$LYCornUnits+.01))
#test$LYBeanUnits=log(abs(test$LYBeanUnits+.01))
#test$LYWheatUnits=log(abs(test$LYWheatUnits+.01))
#test$AT28=log(abs(test$AT28+.01))
#test$AT33=log(abs(test$AT33+.01))
#test$AT36=log(abs(test$AT36+.01))
#test$BC33=log(abs(test$BC33+.01))
#test$BC98=log(abs(test$BC98+.01))
#test$MT28=log(abs(test$MT28+.01))
#test$MT36=log(abs(test$MT36+.01))
#test$RE34=log(abs(test$RE34+.01))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(e

In [5]:
#Census Wrangling

census_api_key("0e29c36736630228c3e7f39191458ccad51de0e1")
v17 <- load_variables(2017, "acs5", cache = TRUE)
#View(v17)

To install your API key for use in future sessions, run this function with `install = TRUE`.



if(FALSE){
    
# median household income
z <- get_acs(geography = "zcta", variables = c(houseIncome="B19013_001")
z$Zip_Code = z$GEOCODE
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$estimate[is.na(train$estimate)]=median(train$estimate[!is.na(train$estimate)])
train$houseIncome = train$estimate

#Age
z_2 <- get_acs(geography = "zcta", variables = "B01002_001E")
#Income
z_3 <- get_acs(geography = "zcta", variables = "B06011_001")
#num males
z_3 <- get_acs(geography = "zcta", variables = "B01001_002")
head(z_3)
#num females
z_3 <- get_acs(geography = "zcta", variables = "B01001_026")
head(z_3)
#median age
z_3 <- get_acs(geography = "zcta", variables = "B01002_001")
head(z_3)
#median age male
z_3 <- get_acs(geography = "zcta", variables = "B01002_002")
head(z_3)
#median age female
z_3 <- get_acs(geography = "zcta", variables = "B01002_003")
head(z_3)

             
             
#Car drove alone
z_3 <- get_acs(geography = "zcta", variables = "B08101_009")
head(z_3)
#Carpooled
z_3 <- get_acs(geography = "zcta", variables = "B08101_017")
head(z_3)
#Public Transportation
z_3 <- get_acs(geography = "zcta", variables = "B08006_008")
head(z_3)
#Num house with mortgages
z_3 <- get_acs(geography = "zcta", variables = "B25027_002")
head(z_3)
#Num house without mortgages
z_3 <- get_acs(geography = "zcta", variables = "B25027_010")
head(z_3)
#Mortgage status
z_3 <- get_acs(geography = "zcta", variables = "B25081_001")
head(z_3)
             
#male 25-34 bachelors degree
z_3 <- get_acs(geography = "zcta", variables = "B15001_017")
head(z_3)
#female 25-34 bachelors degree
z_3 <- get_acs(geography = "zcta", variables = "B15001_058")
head(z_3)
#total bachelor degrees
z_3 <- get_acs(geography = "zcta", variables = "B15011_001")
head(z_3)
#high school diploma
z_3 <- get_acs(geography = "zcta", variables = "B15003_017")
head(z_3)
#less than high school
z_3 <- get_acs(geography = "zcta", variables = "B16010_002")
head(z_3)
#total masters
z_3 <- get_acs(geography = "zcta", variables = "B15003_023")
head(z_3)
#total doctorate
z_3 <- get_acs(geography = "zcta", variables = "B15003_025")
head(z_3)
             
}

In [6]:
#HouseHold Income
z <- get_acs(geography = "zcta", variables = "B19013_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,houseIncome = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
hincome = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$houseIncome[is.na(train$houseIncome)]=median(train$houseIncome[!is.na(train$houseIncome)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$houseIncome[is.na(test$houseIncome)]=median(test$houseIncome[!is.na(test$houseIncome)])

#Age
z <- get_acs(geography = "zcta", variables = "B01002_001E")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,age = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$age[is.na(train$age)]=median(train$age[!is.na(train$age)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$age[is.na(test$age)]=median(test$age[!is.na(test$age)])

#Income
z <- get_acs(geography = "zcta", variables = "B06011_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,singIncome = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$singIncome[is.na(train$singIncome)]=median(train$singIncome[!is.na(train$singIncome)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$singIncome[is.na(test$singIncome)]=median(test$singIncome[!is.na(test$singIncome)])

#num males
z <- get_acs(geography = "zcta", variables = "B01001_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nMale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nMale[is.na(train$nMale)]=median(train$nMale[!is.na(train$nMale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nMale[is.na(test$nMale)]=median(test$nMale[!is.na(test$nMale)])

#num femaels
z <- get_acs(geography = "zcta", variables = "B01001_026")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nFemale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nFemale[is.na(train$nFemale)]=median(train$nFemale[!is.na(train$nFemale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nFemale[is.na(test$nFemale)]=median(test$nFemale[!is.na(test$nFemale)])

#median age
z <- get_acs(geography = "zcta", variables = "B01002_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAge = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
age = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAge[is.na(train$medAge)]=median(train$medAge[!is.na(train$medAge)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAge[is.na(test$medAge)]=median(test$medAge[!is.na(test$medAge)])

#med Age male
z <- get_acs(geography = "zcta", variables = "B01002_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAgeMale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAgeMale[is.na(train$medAgeMale)]=median(train$medAgeMale[!is.na(train$medAgeMale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAgeMale[is.na(test$medAgeMale)]=median(test$medAgeMale[!is.na(test$medAgeMale)])

#median age femal
z <- get_acs(geography = "zcta", variables = "B01002_003")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAgeFemale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAgeFemale[is.na(train$medAgeFemale)]=median(train$medAgeFemale[!is.na(train$medAgeFemale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAgeFemale[is.na(test$medAgeFemale)]=median(test$medAgeFemale[!is.na(test$medAgeFemale)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [7]:
#Car drove alone
z <- get_acs(geography = "zcta", variables = "B08101_009")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cAlone = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cAlone[is.na(train$cAlone)]=median(train$cAlone[!is.na(train$cAlone)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cAlone[is.na(test$cAlone)]=median(test$cAlone[!is.na(test$cAlone)])

#Carpooled
z <- get_acs(geography = "zcta", variables = "B08101_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cPool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cPool[is.na(train$cPool)]=median(train$cPool[!is.na(train$cPool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cPool[is.na(test$cPool)]=median(test$cPool[!is.na(test$cPool)])

#public transportation
z <- get_acs(geography = "zcta", variables = "B08006_008")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cPub = estimate)
pub = z
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cPub[is.na(train$cPub)]=median(train$cPub[!is.na(train$cPub)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cPub[is.na(test$cPub)]=median(test$cPub[!is.na(test$cPub)])


#num Mortgages
z <- get_acs(geography = "zcta", variables = "B25027_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nMort = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
mort = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nMort[is.na(train$nMort)]=median(train$nMort[!is.na(train$nMort)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nMort[is.na(test$nMort)]=median(test$nMort[!is.na(test$nMort)])

#num no Mortgages
z <- get_acs(geography = "zcta", variables = "B25027_010")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nNoMort = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nNoMort[is.na(train$nNoMort)]=median(train$nNoMort[!is.na(train$nNoMort)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nNoMort[is.na(test$nNoMort)]=median(test$nNoMort[!is.na(test$nNoMort)])

#Mortgage Status
z <- get_acs(geography = "zcta", variables = "B25081_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,mortStat = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$mortStat[is.na(train$mortStat)]=median(train$mortStat[!is.na(train$mortStat)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$mortStat[is.na(test$mortStat)]=median(test$mortStat[!is.na(test$mortStat)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [8]:
#male 25-34 bachelors degree
z <- get_acs(geography = "zcta", variables = "B15001_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,mBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$mBach[is.na(train$mBach)]=median(train$mBach[!is.na(train$mBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$mBach[is.na(test$mBach)]=median(test$mBach[!is.na(test$mBach)])

#female 25-34 bachelors degree
z <- get_acs(geography = "zcta", variables = "B15001_058")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,fBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$fBach[is.na(train$fBach)]=median(train$fBach[!is.na(train$fBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$fBach[is.na(test$fBach)]=median(test$fBach[!is.na(test$fBach)])

#total Bachelors degrees
z <- get_acs(geography = "zcta", variables = "B15011_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,totBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$totBach[is.na(train$totBach)]=median(train$totBach[!is.na(train$totBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$totBach[is.na(test$totBach)]=median(test$totBach[!is.na(test$totBach)])

#high school diploma
z <- get_acs(geography = "zcta", variables = "B15003_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,hSchool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
skool = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$hSchool[is.na(train$hSchool)]=median(train$hSchool[!is.na(train$hSchool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$hSchool[is.na(test$hSchool)]=median(test$hSchool[!is.na(test$hSchool)])

#less than high school
z <- get_acs(geography = "zcta", variables = "B16010_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,noSchool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$noSchool[is.na(train$noSchool)]=median(train$noSchool[!is.na(train$noSchool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$noSchool[is.na(test$noSchool)]=median(test$noSchool[!is.na(test$noSchool)])

#total masters
z <- get_acs(geography = "zcta", variables = "B15003_023")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,master = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$master[is.na(train$master)]=median(train$master[!is.na(train$master)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$master[is.na(test$master)]=median(test$master[!is.na(test$master)])

#total doctorate
z <- get_acs(geography = "zcta", variables = "B15003_025")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,doc = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$doc[is.na(train$doc)]=median(train$doc[!is.na(train$doc)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$doc[is.na(test$doc)]=median(test$doc[!is.na(test$doc)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [9]:
data(zipcode)
#zipcode

zipcode$Zip_Code = as.numeric(zipcode$zip)

head(zipcode)

dim(train)
train = merge(x=train,y=zipcode,by = "Zip_Code",all.x=TRUE)
test = merge(x=test,y=zipcode,by = "Zip_Code",all.x=TRUE)
#Make sure I did that rite
dim(train)

#We get some null lats + longs - fix it

train$latitude[is.na(train$latitude)]=median(train$latitude[!is.na(train$latitude)])
train$longitude[is.na(train$longitude)]=median(train$longitude[!is.na(train$longitude)])
test$latitude[is.na(test$latitude)]=median(test$latitude[!is.na(test$latitude)])
test$longitude[is.na(test$longitude)]=median(test$longitude[!is.na(test$longitude)])


,zip,city,state,latitude,longitude,Zip_Code
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,00210,Portsmouth,NH,43.0059,-71.0132,210
2,00211,Portsmouth,NH,43.0059,-71.0132,211
3,00212,Portsmouth,NH,43.0059,-71.0132,212
4,00213,Portsmouth,NH,43.0059,-71.0132,213
5,00214,Portsmouth,NH,43.0059,-71.0132,214
6,00215,Portsmouth,NH,43.0059,-71.0132,215


[1] 99517    47

[1] 99517    52

In [10]:
#aggregate num loans per zip
num_zip = train %>% count(Zip_Code)
head(num_zip)

train = merge(x=train,y=num_zip,by = "Zip_Code",all.x=TRUE)
test = merge(x=test,y=num_zip,by = "Zip_Code",all.x=TRUE)
test$n[is.na(test$n)]=median(test$n[!is.na(test$n)])

Zip_Code,n
<dbl>,<int>
2139,2
5067,2
10004,1
12010,1
13069,2
13685,8


In [11]:
#split train into train/test subsets
datum_train = sample( nrow(train) , as.integer(nrow(train)*.8) )
train_1 = train[datum_train,]
train_2 = train[-datum_train,]

head(train)

,Zip_Code,CreditScore,TYCornUnits,TYBeanUnits,TYWheatUnits,LYCornUnits,LYBeanUnits,LYWheatUnits,State,AT28,⋯,hSchool,noSchool,master,doc,zip,city,state,latitude,longitude,n
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>
1,2139,740,180,495,135,90,1862,145,MA,2800,⋯,1700,1319,6998,3287,02139,Cambridge,MA,42.36435,-71.10431,2
2,2139,738,180,495,135,90,1862,145,MA,2800,⋯,1700,1319,6998,3287,02139,Cambridge,MA,42.36435,-71.10431,2
3,5067,715,160,495,135,55,450,145,IA,253311,⋯,25,0,20,15,05067,South Pomfret,VT,43.68751,-72.54272,2
4,5067,665,160,495,135,55,450,145,IA,253311,⋯,25,0,20,15,05067,South Pomfret,VT,43.68751,-72.54272,2
5,10004,614,24,45,135,33,81,145,NY,82674,⋯,50,28,641,81,10004,New York,NJ,40.69923,-74.04118,1
6,12010,672,160,495,135,108,450,145,NY,16600,⋯,5569,2832,1355,63,12010,Amsterdam,NY,42.94064,-74.17899,1


In [12]:
#Regression num loans
model_gbm_7 = gbm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
                       master+doc+G094+MT36+RE34+latitude+longitude,
         data=train_1,n.trees=1000,interaction.depth=7)

Distribution not specified, assuming bernoulli ...


In [13]:
#model_gbm_8 = gbm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
#                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
#                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
#                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
#                       master+doc+G094+MT36+RE34+latitude+longitude,
#         data=train_1,n.trees=1000,interaction.depth=8)

In [14]:
train_2$pred_7 = plogis(predict(model_gbm_7,train_2,n.trees=450))
accuracy(round(train_2$pred_7),train_2$PastDue)
#train_2$pred_8 = plogis(predict(model_gbm_8,train_2,n.trees=500))
#accuracy(round(train_2$pred_8),train_2$PastDue)

threshold,AUC,omission.rate,sensitivity,specificity,prop.correct,Kappa
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.5,0.8569949,0.25,0.75,0.9639899,0.9628718,0.1666046


In [15]:
#Binomail and quasibinomial are the only viable options
#we will use binomial and output a submission here
test$PastDue = plogis(predict(model_gbm_7,test,n.trees=450))
write.csv(test[,c("id","PastDue")],"sub.csv",row.names=F)
